# Apache Hive

# 1 什么是Hive

- Apache Hive是一款建立在Hadoop之上的开源<font color=#FF2121>**数据仓库**</font>系统，可以将存储在Hadoop文件中的<font color=#E36C07>结构化、半结构化数据文件映射为一张数据库表</font>，基于表提供了一种类似SQL的查询模型，称为<font color=#E36C07>Hive查询语言（HQL）</font>，用于访问和分析存储在Hadoop文件的大型数据集。
- Hive核心是将<font color=#FF2121>**HQL转换为MapReduce程序**</font>，然后将程序提交到Hadoop群集执行。
- Hive由Facebook实现并开源。

# 2 为什么使用Hive

- 使用Hadoop MapReduce直接处理数据所面临的的问题
    - 人员学习成本太高、需要掌握Java语言
    - MapReduce实现复杂查询逻辑开发难度太大
- 使用Hive处理数据的好处
    - 操作接口采用类似SQL语法，提供快速开发的能力
    - 避免直接写MapReduce，减少开发人员的学习成本
    - 支持自定义函数，功能扩展很方便
    - 背靠Hadoop，擅长存储分析海量数据集

# 3 Hive和Hadoop关系

- 从功能来说，数据仓库软件，至少需要具备下述两种能力
    - 存储数据的能力
    - 分析数据的能力
- Apache Hive作为一款大数据时代的数据仓库软件，当然也具备上述两种能力。只不过Hive并不是自己实现了上述两种能力，而是借助Hadoop
    -<font color=#FF2121>**Hive利用HDFS存储数据，利用MapReduce查询分析数据。**</font>
- 这样突然发现Hive没啥用，不过是套壳Hadoop罢了。其实不然，Hive最大的魅力在于<font color=#E36C07>用户专与编写HQL，Hive帮您转换为MapReduce程序完成对数据的分析</font>。

# 案例：如何模拟实现Apache Hive的功能
<br>
如果让您设计Hive这款软件，要求能够实现<font color=#E36C07>用户编写sql语句，Hive自动将sql转换为MapReduce程序，处理位于HDFS上的结构化数据</font>。如何实现？
<br>
在HDFS文件系统上有一个文件，路径为/data/china_user.txt,其内容如下：
<br>
1，张三，18，北京<br>
2，李四，25，上海<br>
3，Allen，30，上海<br>
4，王五，15，南京<br>
5，James，45，杭州<br>
6，Tony，26，北京<br>
需求：统计来自于上海年龄大于25岁的用户有多少个？

## 场景目的
- **重点理解下面两点**：
    - Hive能将数据文件映射成为一张表，这个<font color=#FF2121>**映射**</font>是指什么？
        - <font color=#66CCFF>**文件和表之间的对应关系**</font>
    - Hive软件本身到底承担了什么<font color=#FF2121>**功能职责**</font>？
        - <font color=#66CCFF>**SQL语法解析编译成为MapReduce**</font>
- **映射信息记录**
    - <font color=#FF2121>**映射**</font>在数学上称之为一种<font color=#FF2121>**对应关系**</font>，比如y=x+1，对于每一个x值都有与之对应的y值。
    - 在Hive中能够写sql处理的前提是针对表，而不是针对文件，因此需要将<font color=#FF2121>**文件和表之间的对应关系**</font>描述记录清楚。映射信息专业的叫法称之为<font color=#E36C07>元数据信息</font>（元数据是指用来描述数据的数据metadata）。
    - 具体来看，要记录的元数据信息包括：
        - 表对应着哪个文件（位置信息）
        - 表的列对应着文件哪一个字段（顺序信息）
        - 文件字段之间的分隔符是什么
         
<br>
Hadoop HDFS → hdfs://data/china_user.txt → 
<br> 

|**id** |  **name**   | **age**  | **city** |
|  :----:  |  :----:  | :----:  |:----:  |
| 1 | 张三  | 18 | 北京 | 
| 2 | 李四 | 25  | 上海 | 
| 3 | Allen  | 30 | 上海 |  
| 4 | 王五  | 15 | 南京 |
| 5 | James  | 45 | 杭州 |
| 4 | Tony  | 26 | 北京 |

- **SQL语法解析、编译**
    - 用户写完sql之后，hive需要针对sql进行语法校验，并且根据记录的元数据信息解读sql背后的含义，制定执行计划。并且把执行计划转换为MapReduce程序来执行，把执行结果封装返回给用户。


# 4 Hive组件

- <font color=#FF2121>**用户接口**</font>
    - 包括 CLI、JDBC/ODBC、WebGUI。其中，CLI(command line interface)为shell命令行；Hive中的Thrift服务器允许外部客户端通过网络与Hive进行交互，类似于JDBC或ODBC协议。WebGUI是通过浏览器访问Hive。
- <font color=#FF2121>**元数据存储**</font>
    - 通常是存储在关系型数据库如mysql/derby中。Hive中的元数据包括表的名字、表的列和分区及其属性，表的属性（是否为外部表等），表的数据所在目录等。
- <font color=#FF2121>**Driver驱动程序，包括语法解析器、计划编译器、优化器、执行器**</font>
    - 完成 HQL 查询语句从词法分析、语法分析、编译、优化以及查询计划的生成。生成的查询计划存储在 HDFS 中，并在随后有执行引擎调用执行。
- <font color=#FF2121>**执行引擎**</font>
    - Hive本身并不直接处理数据文件。而是通过执行引擎处理。当下Hive支持MapReduce、Tez、Spark3中引擎。

|**JDBC/ODBC** |  **Hive Driver驱动程序** |  |  |  |
|  :----:  |  :----:  | :----:  |:----:  |:----:  |
| command-line interface | Parser解析器  |  |  | 
| Hive Thrift Server | Planner任务计划 |   |  | 
| Hive Web Interface | Execution执行器  | MapReduce<br>Tez<br>Spark | Hadoop YARN |  
|  | Optimizer优化器  |  |  |
|  | MSClient  |  | Metastore元数据存储 | RDBMS | 


# 5 Apache Hive数据模型

## 5.1 Data Model概念

- 数据模型：用来描述数据、组织数据和对数据进行操作，是对现实世界数据特征的描述。
- Hive的数据模型<font color=#E36C07>类似于RDBMS库表结构</font>，此外<font color=#E36C07>还有自己特有</font>模型。
- Hive中的数据可以在粒度级别上分为三类：
    - Table 表
    - Partition 分区
    - Bucket 桶

## 5.2 Databases数据库

- Hive作为一个数据仓库，在结构上积极向传统数据库看齐，也分数据库（Schema）,每个数据库下面有各自的表组成。<font color=#E36C07>默认数据库default</font>。
- Hive的数据都是<font color=#FF2121>**存储在HDFS上**</font>的，默认有一个根目录，在Hive-site.xml中，由参数hive.metastore.warehouse.dir指定。默认值为<font color=#E36C07>/userhive/warehouset</font>。
- 因此，hive中的数据库在HDFS上的存储路径为：${hive.metastore.warehouse.dir}databasename.db
- 比如，名为itcast的数据库存储路径为：/user/hive/warehouse/itcast.db

## 5.3 Tables表

- Hive表与关系数据库中的表相同。Hive中的所对应的数据通常是存储在HDFS中，而表相关的元数据是存储在RDBMS中。
- Hive中的表的数据在HDFS上的存储路径为：${hive.metastore.warehouse.dir}databasename.db/tablename
- 比如，itcast的数据库下t_user表存储路径为：/user/hive/warehouse/itcast.db/t_user

## 5.4 Partition分区

- Partition分区是hive的一种优化手段表。分区是指<font color="FF2121">**根据分区列（例如“日期day”）的值将表划分为不同分区**</font>。这样可以更快地对指定分区数据进行查询。
- 分区在存储层面上的表现是：table表目录下以子文件夹形式存在。
- <font color="FF2121">一个文件夹表示一个分区</font>。子文件命名标准：分区列=分区值
- Hive还支持分区下继续创建分区，所谓的多重分区。

## 5.5 Buckets分桶

- Buckets分桶表是hive的一种优化手段表。分桶是指<font color="FF2121">**根据表中字段（例如“编号ID”）的值，经过hash计算规则将数据文件划分成指定的若干个小文件**</font>。
- 分桶规则：hashfunc（字段）% 桶个数，余数相同的分到同一个文件。
- 分桶的好处是可以<font color=#E36C07>优化join查询和方便抽样查询</font>。
- Buckets分通表在HDFS中表现为<font color=#E36C07>同一个表目录下数据根据hash散列之后变成多个文件</font>。

# 6 Hive和MySQL对比

- Hive虽然具有RDBMS数据库的外表，包括数据模型、SQL语法都十分相似，但应用场景却完全不同。
- <font color="FF2121">**Hive只适合用来做海量数据的离线分析。Hive的定位是数据仓库，面向分析的OLAP系统**</font>。
- 因此时刻告诉自己，<font color="FF2121">**Hive不是大型数据库，也不是要取代MySQL承担业务数据处理**</font>。

| |  **Apache Hive**   | **Mysql**  |
|  :----:  |  :----:  | :----:  |
| **定位** | 数据仓库  | 数据库 |
| **使用场景** | 离线数据分析 | 业务数据事务处理 | 
| **查询语言** | HQL  | SQL |  
| **数据存储** | HDFS  | Local FS |
| **执行引擎** | MR、Tez、Spark  | Excutor |
| **执行延迟** | 高  | 低 |
| **处理数据规模** | 大  | 小 |
| **常见操作** | 导入数据、查询  | 增删改查 |

# 7 什么是元数据

- 元数据（Metadata）
    - 元数据（Metadata），又称中介数据、中继数据，为<font color="FF2121">**描述数据的数据**</font>（data about data），主要是描述数据属性（property）的信息，用来支持如指示存储位置、历史数据、资源查找、文件记录等功能。
- Hive Metadata
    - Hive Metadata即Hive的**元数据**。
    - 包含用Hive创建的database、table、表的位置、类型、属性、字段顺序类型等元信息。
    - <font color=#E36C07>元数据存储在关系型数据库中</font>。如Hive内置的derby、或者第三方如MySQL.
- Hive Metastore
    - Metastore即<font color="FF2121">**元数据服务**</font>。Metastore服务的作用是<font color="FF2121">管理metadata元数据</font>，对外暴露服务地址，让各种客户端通过连接metastore服务，由metastore再去连接MySQL数据库来存取元数据。
    - 有了metastore服务，就可以有多个客户端同时连接，而且这些客户端不需要知道MySQL数据库的用户名和密码，只需要连接metastore 服务即可。某种程度上也保证了hive元数据的安全。

# 8 Metastore配置方式

## 8.1 概述

- metastore服务配置有3中模式
    - 内嵌模式
    - 本地模式
    - 远程模式
- 区分3中配置方式的关键是弄清楚两个问题：
    - Metastore服务是否需要单独配置、单独启动？
    - Metastore是存储在内置的derby中，还是第三方RDBMS，比如MySQL？

| |  **内嵌模式**   | **本地模式**  | **远程模式**  |
|  :----:  |  :----:  | :----:  |:----:  |
| **Metastore单独配置、启动** | 否  | 否 | 是 |
| **Metastore存储介质** | Derby | Mysql | Mysql | 

## 8.2 内嵌模式

- <font color="FF2121">**内嵌模式**</font>（Embedded Metastore）是Metastore默认部署模式。
- 此种模式下，元数据存储在<font color=#E36C07>内置的Derby数据库</font>，并且Derby数据库和Metastore服务都嵌入在主HiveServer进程中，当启动HiveServer进程时，Derby和Metastore都会启动，<font color=#E36C07>不需要额外起Metastore服务</font>。
- 但是一次只能支持一个活动用户，适用于测试体验，不适用于生产环境。
<br>

**Hive Service JVM： Hive Driver → Metastore → Derby**

# 8.3 本地模式

- <font color="FF2121">**本地模式**</font>（Local Metastore）下，<font color=#E36C07>Metastore服务与主HiveServer进程在同一进程中运行</font>，但是存储元数据的数据库在单独的进程中运行，并且可以在单独的主机上。Metastore服务将通过JDBC与Metastore数据库进行通信。
- <font color=#E36C07>本地模式采用外部数据库来存储元数据</font>，推荐使用MySQL。
- hive根据hive.metastore.uris参数值来判断，如果为空，则为本地模式。
- 缺点是：每启动一次hive服务，都内置启动了一个Metastore。
<br>

**Hive Service JVM： Hive Driver → Metastore↘ <br> Hive Service JVM： Hive Driver → Metastore↗
 MySql**

# 8.4 远程模式

- <font color="FF2121">**远程模式**</font>（Remote Metastore）下，<font color=#E36C07>Metastore服务在其自己的单独JVM上运行，而不在HiveServer的JVM中运行</font>，如果其他进程希望与与Metastore服务器通信，则可以使用Thrift Network API进行通信。
- 远程模式下，需要配置hive.metastore.uris参数来指定Metastore服务运行的机器ip和端口，并且<font color=#E36C07>需要单独手动启动Metastore服务</font>，元数据也采用外部数据库来存储元数据，推荐使用MySQL。
- 在生产环境中，建议用远程模式来配置Hive Metastore。在这种情况下，其他依赖hive的软件都可以通过Metastore访问hive。由于还可以完全屏蔽数据库层，因此这也带来了更好的可管理性/安全性。